# Chazam

<img src='images/chazam.png' width='500'/>

In [ ]:
# librerias

from pydub import AudioSegment  # cargar audio

import pylab as plt     # para pintar

import matplotlib.mlab as mlab   # para sacar espectrograma

import numpy as np   # un poco de algebra lineal😁

import warnings    # para no imprimir warnings
warnings.filterwarnings('ignore')

### 01 - Audio

In [ ]:
# carga de audio

nombre = 'Masayoshi Takanaka - Manifestation'

cancion = AudioSegment.from_file('audio/'+ nombre + '.wav')

In [ ]:
cancion

### 02 - Señal (waveform)

In [ ]:
# gráfico de la señal de audio (amplitud vs tiempo)

plt.figure(figsize=(15, 8))

plt.plot(cancion.get_array_of_samples())

plt.title(nombre)

plt.xlabel('Tiempo')

plt.ylabel('Amplitud');

### 03 - Espectrograma

In [ ]:
# extraccion de un solo canal, 2 canales (cancion.channels)

canal = np.frombuffer(cancion.raw_data, np.int16)[0::2]

In [ ]:
canal.shape   # dimension

In [ ]:
canal[int(8e6) : int(8e6) + 10]   # muestra

In [ ]:
# transformada de Fourier, extraccion componentes de frecuencia

espectro = mlab.specgram(
                        canal,
                        NFFT=5000,
                        Fs=48000,
                        window=mlab.window_hanning,
                        noverlap=int(5000 * 0.7)
                        )[0]


espectro.shape

In [ ]:
espectro[0][50:60]

In [ ]:
# transformacion logaritmica, mlab devuelve un array lineal, se excluyes los 0 para evitar warnings (me salen igual🤨)
# para graficar 

espectro = 10 * np.log10(espectro, out=np.zeros_like(espectro), where=(espectro != 0))

espectro[0][50:60]

In [ ]:
# gráfico del espectrograma (frecuencia vs tiempo: ejes. amplitud: color)

fig, ax = plt.subplots(figsize=(20, 8))

ax.imshow(espectro, cmap='jet')

ax.set_xlabel('Tiempo')
ax.set_ylabel('Frecuencia')

ax.set_title(f'Espectrograma - {nombre}')

plt.gca().invert_yaxis();

![eq](images/eq.gif)

### 04 - Extraer picos en el espectrograma

In [ ]:
from chazam.tools import get_spectrum_peaks

In [ ]:
picos = get_spectrum_peaks(espectro)

len(picos)

In [ ]:
picos[0]

In [ ]:
# espectrograma con picos

fig, ax = plt.subplots(figsize=(20, 8))

ax.imshow(espectro, cmap='jet')

ax.scatter([e[1] for e in picos], [e[0] for e in picos], s=10)

ax.set_xlabel('Tiempo')
ax.set_ylabel('Frecuencia')

ax.set_title(f'Espectrograma - {nombre}')

plt.gca().invert_yaxis();

In [ ]:
# picos

fig, ax = plt.subplots(figsize=(20, 8))

ax.scatter([e[1] for e in picos], [e[0] for e in picos], s=15)

ax.set_xlabel('Tiempo')
ax.set_ylabel('Frecuencia')

ax.set_title(f'Picos - {nombre}');

### 05 - Hashing fingerprints

<img src='images/fingerprint.jpg' width='500'/>

[funcion hash](https://es.wikipedia.org/wiki/Funci%C3%B3n_hash)


$hashing(\text{picos}, \text{diferencia tiempo entre picos}) = \text{valor hash del fingerprint, tiempo inicial}$

In [ ]:
from chazam.tools import hashing

In [ ]:
hash=hashing(picos, 15)

hash[-10:]

In [ ]:
len(hash)

### 06 - Base de datos (SQL)

**Creación tabla de canciones**

```sql
create table if not exists songs (
    
                                song_id mediumint unsigned not null auto_increment,
                                song_name varchar(250) not null,
                                fingerprinted tinyint default 0,
                                file_sha1 binary(20) not null,
    
                    primary key (song_id),
                    unique key song_id (song_id)
                );
```

**Creacion tabla de fingerprints**

```sql
create table if not exists fingerprints (
                                         
                                         hash binary(10) not null,
                                         song_id mediumint unsigned not null,
                                         offset int unsigned not null,
                     index (hash),
                     unique key `unique_constraint` (song_id, offset, hash),
                     foreign key (song_id) references fingerprints(song_id) on delete cascade
                );
```

**Insercion de datos en tabla canciones**

```sql
insert into songs (song_name, file_sha1) values (song_name_data, UNHEX(file_sha1_data));
```

**Insercion de datos en tabla fingerprints**

```sql
insert ignore into fingerprints (hash, song_id, offset) values (UNHEX(hash), song_id, offset);
```

### 07 - Detección

In [ ]:
from chazam import Chazam

from chazam.tools import audio_cutting

In [ ]:
# recorte de audio a detectar desde 1:50 hasta 1:53

audio_cutting(nombre, 1, 50, 1, 53)

In [ ]:
AudioSegment.from_file('sample/'+ nombre + '.wav')

In [ ]:
ih_chazam=Chazam()

In [ ]:
cancion=ih_chazam.recognize('sample/'+ nombre + '.wav')

cancion

In [ ]:
cancion['results'][0]['song_name'].decode('utf-8')

### 08 - API (deploy)

```python

from flask import Flask, request
from flask_cors import CORS        # cross-origin resource sharing

from chazam import Chazam

from config import WAV_PATH, COVERS, DATA_DEFAULT


app=Flask(__name__)

cors=CORS(app, resources={r'*': {'origins': '*'}})

ih_chazam=Chazam()


@app.route('/', methods=['POST', 'GET'])
def main():

    audio=request.files['blob']
    audio.save(WAV_PATH)

    song=ih_chazam.recognize(WAV_PATH)

    if song['results'][0]['input_confidence']<0.2:
        return f'{DATA_DEFAULT}', 200

    else:

        song_name=song['results'][0]['song_name'].decode('utf-8').split('-')

        data={'artist': song_name[0].strip(),
              'song': song_name[1].strip(),
              'cover': COVERS[song['results'][0]['song_name'].decode('utf-8')],
              'match_time': song['total_time']
              }

        return f'{data}', 200

   
if __name__== '__main__':
    app.run(debug=True)




```